<a href="https://colab.research.google.com/github/Earth15145/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

# ข้อ1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MIDterm')

!pwd # เช็ค path ที่กำลังทำงานอยู่

/content/drive/MyDrive/MIDterm


In [ ]:
house_data = pd.read_csv("/content/drive/MyDrive/MIDterm/Data House.csv")

In [ ]:
house_data

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# prompt: สนใจแค่ 'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone'

# Select only the specified columns
selected_columns = ['renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story',
                    'number_of_bedroom','number_of_bathroom','direction','furniture','usable_area',
                    'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']
house_data_selected = house_data[selected_columns]
house_data_selected

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN
1,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,น้ำพอง,ทรายมูล,NaN
2,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN
3,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,no,0,0,73.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,108.76,NaN,437400.0,บ้านฝาง,หนองบัว,NaN
1652,no,0,0,84.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,119.00,NaN,714600.0,หนองเรือ,โนนทอง,NaN
1653,no,0,1,45.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,188.50,NaN,451000.0,แวงน้อย,ก้านเหลือง,NaN
1654,no,0,1,59.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,207.00,NaN,910000.0,ชุมแพ,วังหินลาด,NaN


In [ ]:
# prompt: zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

# Group data by 'zone' and calculate the mean 'sell_price' for each zone
zone_avg_price = house_data_selected.groupby('zone')['sell_price'].mean()

# Find the zone with the highest average 'sell_price'
max_avg_price_zone = zone_avg_price.idxmax()

print(f"Zone with the highest average selling price: {max_avg_price_zone}")

Zone with the highest average selling price: ในเมือง-รื่นรมย์-เทพารักษ์


# ข้อ2

In [44]:
import pandas as pd
import json
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Load data
house_data = pd.read_csv("/content/drive/MyDrive/MIDterm/Data House.csv")
with open('/content/drive/MyDrive/MIDterm/7-eleven.json', 'r') as f:
    seven_eleven_data = json.load(f)
with open('/content/drive/MyDrive/MIDterm/school.json', 'r') as f:
    school_data = json.load(f)

# Initialize columns
house_data['d1000-7/11'] = 0
house_data['d1000-school'] = 0

# Process each house
for index, row in house_data.iterrows():
    house_lat = row['latitude']
    house_lon = row['longitude']

    seven_eleven_count = 0
    school_count = 0

    # Count 7-11 within 1km
    # Iterate through the values in the seven_eleven_data dictionary
    for seven_eleven_list in seven_eleven_data.values():
        #  Check if the list is not empty to account for the locations with an empty list
        if seven_eleven_list:
            # Loop through each location in the list
            for seven_eleven in seven_eleven_list:
                # Check if 'longitude' and 'latitude' keys exist before accessing them
                if 'longitude' in seven_eleven and 'latitude' in seven_eleven:
                    distance = haversine(house_lon, house_lat, seven_eleven['longitude'], seven_eleven['latitude'])
                    if distance <= 1:
                        seven_eleven_count += 1

    # Count schools within 1km
    # Iterate through the values in the school_data dictionary
    for school_list in school_data.values():
        # Check if the list is not empty to account for the locations with an empty list
        if school_list:
            # Loop through each location in the list
            for school in school_list:
                # Check if 'longitude' and 'latitude' keys exist before accessing them
                if 'longitude' in school and 'latitude' in school:
                    distance = haversine(house_lon, house_lat, school['longitude'], school['latitude'])
                    if distance <= 1:
                        school_count += 1

    house_data.loc[index, 'd1000-7/11'] = seven_eleven_count
    house_data.loc[index, 'd1000-school'] = school_count

# Display the updated dataframe (optional)
print(house_data[['id', 'd1000-7/11', 'd1000-school']].head())

   id  d1000-7/11  d1000-school
0   1           0             0
1   4           0             0
2  16           0             0
3  18           0             0
4  19           0             0


In [42]:
# prompt: บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

# Find the house with the maximum number of 7-11 within 1km
max_711_house = house_data['d1000-7/11'].max()
max_711_price = house_data[house_data['d1000-7/11'] == max_711_house]['sell_price'].iloc[0]

print(f"The house with the most 7-11s within 1km has a price of: {max_711_price}")

The house with the most 7-11s within 1km has a price of: 390000.0


# ข้อ3

In [8]:
answer_data = pd.read_csv("/content/drive/MyDrive/MIDterm/คำตอบนักประเมิน(ID).csv")

In [9]:
answer_data

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,"965,000","550,000","ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,000,000","ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,"10,873,000","10,000,000","ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,200,000",ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [17]:
# prompt: merge house_data and answer_data โดยเปลี่ยน ID ให้ตรงกัน

import pandas as pd
import numpy as np
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/MIDterm')

house_data = pd.read_csv("/content/drive/MyDrive/MIDterm/Data House.csv")
answer_data = pd.read_csv("/content/drive/MyDrive/MIDterm/คำตอบนักประเมิน(ID).csv")

# Assuming 'ID' column exists in both DataFrames and needs to be used for merging
# Rename the ID column in answer_data to match house_data (if necessary)
answer_data = answer_data.rename(columns={'ID': 'id'})


# Merge the two dataframes based on the 'id' column
merged_data = pd.merge(house_data, answer_data, on='id', how='left')

# Display the merged data
merged_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,name_th,name_en,project_name,company,province,district,sub_district,property_type_x,home improvement,...,has_garden,ID ผู้ประเมิน,post id,property_type_y,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,NaN,1.0,https://nayoo.co/khonkaen/posts/1,บ้านเดี่ยว,มั่นใจ,"390,000","600,000","3,000-30,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
1,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,NaN,2.0,https://nayoo.co/khonkaen/posts/1,บ้านเดี่ยว,มั่นใจ,"390,000","1,300,000","ราคาเสนอขาย 1,000 - 3,000 บาท / ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,NaN,1.0,https://nayoo.co/khonkaen/posts/4,บ้านเดี่ยว,มั่นใจ,"930,000","750,000","2,000-10,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
3,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,NaN,2.0,https://nayoo.co/khonkaen/posts/4,บ้านเดี่ยว,มั่นใจ,"930,000","1,000,000","ราคาเสนอขาย 3,000 - 5,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
4,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,NaN,1.0,https://nayoo.co/khonkaen/posts/16,บ้านเดี่ยว,มั่นใจ,"2,994,000","2,800,000","2,000-10,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี


In [29]:
# Extract the first letter of the 'score' column
merged_data['score'] = merged_data['ให้คะแนนทรัพย์'].astype(str).str[0]
merged_data['estimator'] = merged_data['ประเมินราคาซื้อ-ขาย']

merged_data

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type_x,home improvement,...,post id,property_type_y,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,score,estimator
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,https://nayoo.co/khonkaen/posts/1,บ้านเดี่ยว,มั่นใจ,"390,000","600,000","3,000-30,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4,"600,000"
1,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,https://nayoo.co/khonkaen/posts/1,บ้านเดี่ยว,มั่นใจ,"390,000","1,300,000","ราคาเสนอขาย 1,000 - 3,000 บาท / ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4,"1,300,000"
2,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,https://nayoo.co/khonkaen/posts/4,บ้านเดี่ยว,มั่นใจ,"930,000","750,000","2,000-10,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,3,"750,000"
3,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,https://nayoo.co/khonkaen/posts/4,บ้านเดี่ยว,มั่นใจ,"930,000","1,000,000","ราคาเสนอขาย 3,000 - 5,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4,"1,000,000"
4,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,https://nayoo.co/khonkaen/posts/16,บ้านเดี่ยว,มั่นใจ,"2,994,000","2,800,000","2,000-10,000 บาท/ตารางวา",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,2,"2,800,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n,NaN
2367,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n,NaN
2368,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n,NaN
2369,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n,NaN


In [35]:
# prompt: ของID ผู้ประเมิน = 1 เท่านั้น score	estimator

# Filter for assessor ID = 1
assessor_1_data = merged_data[merged_data['ID ผู้ประเมิน'] == 1]

# Select only the 'score' and 'estimator' columns
assessor_1_scores = assessor_1_data[['score', 'estimator']]

# Display the result
assessor_1_scores

,score,estimator
0,4,"600,000"
2,3,"750,000"
4,2,"2,800,000"
6,2,"4,500,000"
9,2,"3,600,000"
...,...,...
2350,2,"4,500,000"
2351,2,"3,500,000"
2352,2,"1,800,000"
2353,2,"5,500,000"


In [45]:
# prompt: ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

# Group the data by sub_district and calculate the mean score
subdistrict_avg_score = merged_data.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest average score
max_avg_score_subdistrict = subdistrict_avg_score.idxmax()

print(f"Subdistrict with the highest average score: {max_avg_score_subdistrict}")

Subdistrict with the highest average score: ศรีบุญเรือง
